In [1]:
''' 
Script para descargar un archivo de Google Drive y mostrar las primeras 10 líneas
'''

url_view = "https://drive.google.com/file/d/1RMKdCJKS7vFBhiTLosCotau1k4UsN5iu/view?usp=sharing"   # Enlace de Google Drive
file_id = url_view.split('/')[5]  # Extrae el ID del archivo (la 6ª parte del enlace)
download_url = f"https://drive.google.com/uc?id={file_id}"   # Crea el enlace de descarga directa

import requests
response = requests.get(download_url)

# Muestra las primeras 10 líneas, para saber el tipo de separación del csv
for i, line in enumerate(response.text.splitlines()[:10], start=1):
    print(f"{i:02d}: {line}")


01: Date;Year;Type;Country;State;Location;Activity;Name;Sex;Age;Injury;Fatal Y/N;Time;Species ;Source;pdf;href formula;href;Case Number;Case Number;original order;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
02: 11th October;2025;Unprovoked;Australia;Queensland;Cook Esplanade Thursday Island;Fishing/swimming;Samuel Nai;M;14;Serious abdonminal injuries;N;1823 hrs;Tiger or Bull shark;Kevin McMurray Trackingsharks.com;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
03: 7th October;2025;Unprovoked;Australia;South Australia;Kangaroo Island;Surfing;Lee Berryman;M;50+;Lacerations to calf ;N;1330hrs;Bronze whaler?;Kevin

In [2]:
import pandas as pd
from typing import Optional, Iterable

shark_df = pd.read_csv(download_url, sep=';', encoding='utf-8', low_memory=False)
print("Inicialmente los datos tienen:", shark_df.shape)
print(shark_df.head())  # Muestra las primeras filas del DataFrame

shark_df = shark_df.dropna(axis=1, how='all') # Eliminar columnas vacías enteras
print("Ahora los datos tienen:",shark_df.shape)
print(shark_df.columns)  # Muestra las columnas del DataFrame: 255 col a 23 col

Inicialmente los datos tienen: (39139, 255)
             Date  Year        Type     Country            State  \
0    11th October  2025  Unprovoked   Australia       Queensland   
1     7th October  2025  Unprovoked   Australia  South Australia   
2  29th September  2025  Unprovoked         USA   Off California   
3  27th September  2025    Provoked  Costa Rica              NaN   
4   6th September  2025  Unprovoked   Australia              NSW   

                         Location               Activity                Name  \
0  Cook Esplanade Thursday Island       Fishing/swimming          Samuel Nai   
1                 Kangaroo Island                Surfing        Lee Berryman   
2                 Catalina Island               Swimming  Christopher Murray   
3                   Cocos Islands  Diving-Tagging sharks  Dr. Mauricio Hoyos   
4                Long Reef Sydney                Surfing  Mercury Psillaskis   

  Sex  Age  ... Unnamed: 245 Unnamed: 246 Unnamed: 247 Unnamed: 24

In [3]:
#Quitamos las columnas que no nos sirven para el análisis

shark_df.drop(['Date','Location','Name','Age','Injury','Time','Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'], axis=1, inplace=True)

print(shark_df.columns)  # Muestra las columnas del DataFrame después de eliminar las que no analizaremos

Index(['Year', 'Type', 'Country', 'State', 'Activity', 'Sex', 'Fatal Y/N',
       'Species '],
      dtype='object')


In [4]:
""""
Normalize text columns (strip spaces, set lowercase).

Parameters
----------
df : DataFrame
cols : columns to normalize; if None, all object/string columns
lower : convert to lowercase
strip : strip leading/trailing whitespace
normalize_columns: normalize column names (strip, lower, replace spaces with _)

Returns
-------
DataFrame (same object, modified in place style but returns df for chaining)
"""

def standardize_text(
    df: pd.DataFrame,
    cols: Optional[Iterable[str]] = None,
    lower: bool = True,
    strip: bool = True,
    normalize_columns: bool = True,
) -> pd.DataFrame:
       
    if cols is None:
        cols = df.select_dtypes(include=["object", "string"]).columns

    for c in cols:
        s = df[c].astype("string")
        if strip:
            s = s.str.strip()
        if lower:
            s = s.str.lower()
        df[c] = s
     
    if normalize_columns:
        df.columns = (
            df.columns
            .str.strip()
            .str.lower()
            .str.replace(" ", "_")
        )

    return df

shark_df = standardize_text(shark_df)
print(shark_df.head())  # Muestra las primeras filas del DataFrame después de la normalización del texto y las columnas

   year        type     country            state               activity sex  \
0  2025  unprovoked   australia       queensland       fishing/swimming   m   
1  2025  unprovoked   australia  south australia                surfing   m   
2  2025  unprovoked         usa   off california               swimming   m   
3  2025    provoked  costa rica             <NA>  diving-tagging sharks   m   
4  2025  unprovoked   australia              nsw                surfing   m   

  fatal_y/n              species  
0         n  tiger or bull shark  
1         n       bronze whaler?  
2         n   unknown 1.2m shark  
3         n       tiger shark 4m  
4         y    great white shark  


In [5]:
#Check valores únicos por columna:

for col in shark_df.columns:
    print(f'{col.upper()} --> valores únicos:', shark_df[col].unique())

YEAR --> valores únicos: <StringArray>
['2025', '2024', '2026', '2023', '2022', '2021', '2020', '2019', '2018',
 '2017',
 ...
 '1580', '1555', '1554', '1543', '1518', '1500', '1000', '0077', '0005',
 '0000']
Length: 263, dtype: string
TYPE --> valores únicos: <StringArray>
[         'unprovoked',            'provoked',        'questionable',
          'watercraft',        'sea disaster',                  <NA>,
                   '?',         'unconfirmed',          'unverified',
             'invalid', 'under investigation',                'boat']
Length: 12, dtype: string
COUNTRY --> valores únicos: <StringArray>
[               'australia',                      'usa',
               'costa rica',                  'bahamas',
              'puerto rico',         'french polynesia',
                    'spain',           'canary islands',
             'south africa',                  'vanuatu',
 ...
        'mediterranean sea',                   'sweden',
                   'roatan', 'b

In [6]:
shark_df.dtypes

year         string[python]
type         string[python]
country      string[python]
state        string[python]
activity     string[python]
sex          string[python]
fatal_y/n    string[python]
species      string[python]
dtype: object

In [7]:
print('Los valores iniciales eran:', shark_df['year'].nunique)
print('Los valores iniciales eran:', shark_df['year'].shape)
print('Los valores iniciales eran:', shark_df['type'].nunique)
print('Los valores iniciales eran:', shark_df['type'].shape)

#PRIMER FILTRO (Marlene):

shark_df = shark_df[shark_df['type'] == 'unprovoked'].copy() #Quitamos los accidentes que no sean "unprovoked"

Los valores iniciales eran: <bound method IndexOpsMixin.nunique of 0        2025
1        2025
2        2025
3        2025
4        2025
         ... 
39134    <NA>
39135    <NA>
39136    <NA>
39137    <NA>
39138    <NA>
Name: year, Length: 39139, dtype: string>
Los valores iniciales eran: (39139,)
Los valores iniciales eran: <bound method IndexOpsMixin.nunique of 0        unprovoked
1        unprovoked
2        unprovoked
3          provoked
4        unprovoked
            ...    
39134          <NA>
39135          <NA>
39136          <NA>
39137          <NA>
39138          <NA>
Name: type, Length: 39139, dtype: string>
Los valores iniciales eran: (39139,)


In [8]:
#Exportamos a excel post filtro: TYPE

shark_df.to_excel('SharkAttack_check1.xlsx')

In [9]:
print(shark_df[['year']].isnull().sum())

shark_df = shark_df.dropna(subset=['year']) #Quitamos los null de la columna "year"

print(shark_df.isnull().sum())

year    1
dtype: int64
year            0
type            0
country        33
state         300
activity      359
sex           168
fatal_y/n      17
species      2598
dtype: int64


In [10]:
#formateamos para que todos los string sean igual (con decimal ".")

shark_df['year'] = (
    shark_df['year']
    .astype(str)                 
    .str.replace(',', '.', regex=False)  # cambia coma por punto
    .astype(float)      
)
shark_df['year'] = pd.to_numeric(shark_df['year'])
shark_df = shark_df.loc[(shark_df['year'] >= 2000) & (shark_df['year'] <= 2025)] #filtramos los años sujetos a estudio

#CHECK FILTRO 1
print('Post filtro quedan:', shark_df['year'].shape , "valores")
print('Post filtro quedan:', shark_df['type'].shape , "valores")


Post filtro quedan: (2206,) valores
Post filtro quedan: (2206,) valores


In [11]:
#Check valores por columnas para próximos filtros

for col in shark_df.columns:
    print(f'{col.upper()} --> valores únicos:', shark_df[col].unique())

YEAR --> valores únicos: [2025. 2024. 2023. 2022. 2021. 2020. 2019. 2018. 2017. 2016. 2015. 2014.
 2013. 2012. 2011. 2010. 2009. 2008. 2007. 2006. 2005. 2004. 2003. 2002.
 2001. 2000.]
TYPE --> valores únicos: <StringArray>
['unprovoked']
Length: 1, dtype: string
COUNTRY --> valores únicos: <StringArray>
[                            'australia',
                                   'usa',
                               'bahamas',
                           'puerto rico',
                      'french polynesia',
                                 'spain',
                        'canary islands',
                          'south africa',
                               'vanuatu',
                               'jamaica',
                                'israel',
                              'maldives',
                      'turks and caicos',
                            'mozambique',
                         'new caledonia',
                                 'egypt',
                      

In [12]:
#Exportamos a excel post filtro: TYPE + YEAR

shark_df.to_excel('SharkAttack_check2.xlsx')

In [13]:
#SEGUNDO FILTRO (Marta):
#Decidimos filtrar por 11 países que representan el 90% de los ataques. 
country_selected = [
    "usa","australia","south africa","bahamas","brazil","new zealand",
    "new caledonia","egypt","reunion","french polynesia","mexico","reunion island"]

shark_df_filtered = shark_df[
    shark_df["country"].str.strip().str.lower().isin(country_selected)
].copy()

print("Después del segundo filtro, los datos tienen:", shark_df_filtered.shape)
# Reunion y Reunion island los llamamos como Reunion Island

map_reunion = {"reunion island": "reunion"}
shark_df_filtered["country"] = shark_df_filtered["country"].replace(map_reunion)
print(shark_df_filtered["country"].value_counts(dropna=False).sort_index())
shark_df = shark_df_filtered #sobrescribe el dataframe con los filtros y demás


Después del segundo filtro, los datos tienen: (1977, 8)
country
australia            434
bahamas               69
brazil                54
egypt                 30
french polynesia      26
mexico                25
new caledonia         35
new zealand           38
reunion               30
south africa         115
usa                 1121
Name: count, dtype: Int64


In [14]:
# Borramos state en blanco para esos paises
print("Antes:", shark_df_filtered.shape)
shark_df_filtered = shark_df_filtered.dropna(subset=["state"]).copy()
print("Después:", shark_df_filtered.shape)

Antes: (1977, 8)
Después: (1934, 8)


In [15]:
#Normalizando nombres de estados para los países seleccionados
#USA:

mask_usa = shark_df_filtered["country"].eq("usa")

# Diccionario de normalización (todo en minúsculas)
usa_state_map = {
    "floria": "florida",
    "franklin county, florida": "florida",
    "noirth carolina": "north carolina",
    "off california": "california",
    "los angeles": "california",
    "long island ny": "new york",
    "maui": "hawaii",
    "virgin islands": "us virgin islands",
 }

print("Después de normalizar estados USA:", shark_df_filtered.loc[mask_usa, "state"].replace(usa_state_map).value_counts())

Después de normalizar estados USA: state
florida              602
hawaii               152
california           103
south carolina        79
north carolina        69
texas                 34
new york              16
oregon                16
alabama               10
louisiana              6
georgia                5
new jersey             5
massachusetts          4
maine                  2
guam                   2
us virgin islands      2
virginia               2
galveston              1
samoa                  1
maryland               1
cayman islands         1
bahamas                1
washington             1
rhode island           1
delaware               1
palmyra atoll          1
puerto rico            1
johnston atoll         1
Name: count, dtype: Int64


In [16]:
#Australia:

mask_aus = shark_df_filtered["country"].eq("australia")

aus_state_map = {
    "new  south wales": "new south wales",
    "new south ales": "new south wales",
    "nsw": "new south wales",
    "wa": "western australia",
    "westerm australia": "western australia",
    "western  australia": "western australia",
}

print(
    "Después de normalizar estados AU:",
    shark_df_filtered.loc[mask_aus, "state"]
        .replace(aus_state_map)
        .value_counts()
)

Después de normalizar estados AU: state
new south wales                         164
western australia                       120
queensland                               77
south australia                          36
victoria                                 23
tasmania                                  6
northern territory                        5
torres strait                             1
territory of cocos (keeling) islands      1
Name: count, dtype: Int64


In [17]:
#South Africa:
mask_sa = shark_df_filtered["country"].eq("south africa")

sa_state_map = {
    "easten cape province": "eastern cape",
    "eastern cape  province": "eastern cape",
    "eastern cape province": "eastern cape",
    "eastern province": "eastern cape", 
    "kwazulu-natal between port edward and port st johns": "kwazulu-natal",
    "western cape province": "western cape",
    "western province": "western cape",
}

print(
    "Después de normalizar estados South Africa:",
    shark_df_filtered.loc[mask_sa, "state"]
        .replace(sa_state_map)
        .value_counts()
)


Después de normalizar estados South Africa: state
western cape     48
eastern cape     46
kwazulu-natal    19
Name: count, dtype: Int64


In [18]:
#Bahamas:
mask_bhs = shark_df_filtered["country"].eq("bahamas")

bahamas_state_map = {
    # --- Grand Bahama (incluye off-shore y ciudades) ---
    "40 miles off grand bahama island": "grand bahama",
    "grand  bahama island": "grand bahama",
    "grand bahama island": "grand bahama",
    "freeport": "grand bahama",
    "west end": "grand bahama",

    # --- Abaco ---
    "abaco islands": "abaco",
    "great abaco islands": "abaco",
    "atlantic ocean near big grand cay": "abaco",

    # --- Andros ---
    "andros islands": "andros",

    # --- Exuma ---
    "exuma islands": "exuma",
    "exumas": "exuma",
    "the exuma cays": "exuma",

    # --- New Providence (+ Paradise Island y variantes tipográficas) ---
    "new providence   isoad": "new providence",
    "new providence district": "new providence",
    "new providence island": "new providence",
    "paradise island": "new providence",

    # --- Long Island (typo en el dato) ---
    "clarence town long isand": "long island",
}

print(
    "Después de normalizar Bahamas:",
    shark_df_filtered.loc[mask_bhs, "state"]
        .str.strip().str.lower()
        .replace(bahamas_state_map)
        .value_counts()
)

Después de normalizar Bahamas: state
abaco                  22
grand bahama           10
exuma                   7
new providence          7
eleuthera               2
long island             1
lucayan archipelago     1
bimini                  1
northern bahamas        1
andros                  1
Name: count, dtype: Int64


In [19]:
#Brasil:
mask_bra = shark_df_filtered["country"].eq("brazil")

brazil_state_map = {
    "balneário camboriú": "santa catarina",
    "santa catarina state": "santa catarina",
    "rio grande de norte": "rio grande do norte",
    "fernando de noronha": "pernambuco",
    "são paulo.": "sao paulo",
}

print(
    "Después de normalizar Brasil:",
    shark_df_filtered.loc[mask_bra, "state"]
        .str.strip().str.lower()
        .replace(brazil_state_map)
        .value_counts())

Después de normalizar Brasil: state
pernambuco             42
sao paulo               2
santa catarina          2
bahia                   2
rio de janeiro          2
rio grande do sul       1
rio grande do norte     1
Name: count, dtype: Int64


In [20]:
#New Zealand:
mask_nz = shark_df_filtered["country"].eq("new zealand")

nz_state_map = {
    "south island, near karitane north of dunedin": "south island",
    "southland": "south island",
    "bay of waitangi": "north island",
    "mercury islands": "north island",
  }

print(
    "Después de normalizar New Zealand:",
    shark_df_filtered.loc[mask_nz, "state"]
        .str.strip().str.lower()
        .replace(nz_state_map)
        .value_counts()
)

Después de normalizar New Zealand: state
south island       18
north island       16
cook islands        3
chatham islands     1
Name: count, dtype: Int64


In [21]:
#New Caledonia:

mask_nc = shark_df_filtered["country"].eq("new caledonia")

nc_state_map = {
    "bélep islands": "belep islands",
    "grande terre": "grand terre",
}

print(
    "Después de normalizar New Caledonia:",
    shark_df_filtered.loc[mask_nc, "state"]
        .str.strip().str.lower()
        .replace(nc_state_map)
        .value_counts()
)


Después de normalizar New Caledonia: state
south province          8
north province          8
loyalty islands         5
grand terre             2
poum                    1
belep islands           1
noumea                  1
baie de sainte-marie    1
Name: count, dtype: Int64


In [22]:
#Egipto:
mask_egy = shark_df_filtered["country"].eq("egypt")

egypt_state_map = {
    "hurghada, red sea governorate": "red sea governorate",
    "north of marsa alam": "red sea governorate",
    "red sea": "red sea governorate",
    "red sea protectorate": "red sea governorate",
    "st. johns reef": "red sea governorate",
    "sinai peninsula": "south sinai",
    "south sinai peninsula": "south sinai",
}

print(
    "Después de normalizar Egipto:",
    shark_df_filtered.loc[mask_egy, "state"]
        .str.strip().str.lower()
        .replace(egypt_state_map)
        .value_counts()
)


Después de normalizar Egipto: state
red sea governorate    12
south sinai            12
suez                    1
Name: count, dtype: Int64


In [23]:
#Reunion:

mask_reu = shark_df_filtered["country"].eq("reunion")

s = (
    shark_df_filtered.loc[mask_reu, "state"]
        .astype("string")
        .str.strip().str.lower()
        .str.replace("-", " ", regex=False)         # "saint-gilles" -> "saint gilles"
        .replace({
            "saint guilles": "saint gilles",        # typo
            "saint gilles les bains": "saint gilles",
            "d'etang-sale": "etang sale",
            "d’etang-sale": "etang sale",
            "d'etang-salé": "etang sale",
            "d’etang-salé": "etang sale",
            "conservatória district": "conservatoria district",
        })
)

# Agrupa TODO lo que empiece por "saint " en "saint areas"
s = s.where(~s.str.startswith("saint "), "saint areas")

print("Después de normalizar Reunion:")
print(s.value_counts())

Después de normalizar Reunion:
state
saint areas               17
d’étang salé               2
le port                    1
trois bassins              1
bois blanc                 1
conservatoria district     1
Name: count, dtype: Int64


In [24]:
# French Polynesia
mask_fp = shark_df_filtered["country"].eq("french polynesia")

french_poly_map = {
    # Society Islands
    "bora bora": "society islands",
    "moorea": "society islands",
    "tahiti": "society islands",
    "nuku hiva": "marquesas",
    "central tuamotu": "tuamotu islands",
    "tuamotos": "tuamotu islands",
    "tuamotus": "tuamotu islands",
    "rangiroa": "tuamotu islands",
}

print(
    "Después de normalizar French Polynesia:",
    shark_df_filtered.loc[mask_fp, "state"]
        .str.strip().str.lower()
        .replace(french_poly_map)
        .value_counts()
)

Después de normalizar French Polynesia: state
society islands    14
tuamotu islands     6
marquesas           4
gambier islands     1
Name: count, dtype: Int64


In [25]:
#Mexico:
mask_mex = shark_df_filtered["country"].eq("mexico")

mexico_state_map = {
    "guerrero": "guerrero",
    "cabo san lucas": "baja california",
    "baja": "baja california",
    "baja california sur": "baja california",
    "guerro": "guerrero",
}

print(
    "Después de normalizar Méxipwdco:",
    shark_df_filtered.loc[mask_mex, "state"]
        .str.strip().str.lower()
        .replace(mexico_state_map)
        .value_counts()
)

Después de normalizar Méxipwdco: state
quintana roo          7
baja california       6
guerrero              6
sonora                3
jalisco               1
gulf of california    1
sinaloa               1
Name: count, dtype: Int64


In [26]:
#Asignamos los cambios al dataframe principal
# Use shark_df_filtered since the masks were created from it
shark_df_filtered.loc[mask_usa, "state"] = shark_df_filtered.loc[mask_usa, "state"].replace(usa_state_map)
shark_df_filtered.loc[mask_aus, "state"] = shark_df_filtered.loc[mask_aus, "state"].replace(aus_state_map)
shark_df_filtered.loc[mask_sa, "state"] = shark_df_filtered.loc[mask_sa, "state"].replace(sa_state_map)
shark_df_filtered.loc[mask_bhs, "state"] = shark_df_filtered.loc[mask_bhs, "state"].replace(bahamas_state_map)
shark_df_filtered.loc[mask_bra, "state"] = shark_df_filtered.loc[mask_bra, "state"].replace(brazil_state_map)
shark_df_filtered.loc[mask_nz, "state"] = shark_df_filtered.loc[mask_nz, "state"].replace(nz_state_map)
shark_df_filtered.loc[mask_nc, "state"] = shark_df_filtered.loc[mask_nc, "state"].replace(nc_state_map)
shark_df_filtered.loc[mask_egy, "state"] = shark_df_filtered.loc[mask_egy, "state"].replace(egypt_state_map)
shark_df_filtered.loc[mask_reu, "state"] = s
shark_df_filtered.loc[mask_fp, "state"] = shark_df_filtered.loc[mask_fp, "state"].replace(french_poly_map)
shark_df_filtered.loc[mask_mex, "state"] = shark_df_filtered.loc[mask_mex, "state"].replace(mexico_state_map)
shark_df

# Update shark_df to the cleaned version
shark_df = shark_df_filtered


In [27]:
#Exportamos a excel post filtro: TYPE + YEAR + COUNTRY + LOCATION

shark_df.to_excel('SharkAttack_check3.xlsx')